In [22]:
 %load_ext nb_mypy

The nb_mypy extension is already loaded. To reload it, use:
  %reload_ext nb_mypy


In [37]:
import torch
import numpy


In [24]:
import wget

wget.download('https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt')

'input (2).txt'

In [25]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [26]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [50]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [28]:
charToIndex : dict[str, int] = { ch:i for i,ch in enumerate(chars) }
indexToChar : dict[int, str] = { i:ch for i,ch in enumerate(chars) }

def encode(text: str) -> list[int]: 
    return [charToIndex[c] for c in text]

def decode(values: list[int]) -> str:
    return ''.join([indexToChar[value] for value in values])

print(encode("hi there"))
print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [51]:
data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [30]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data  = data[n:]

In [31]:
block_size: int = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [32]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, the target is: {target}")

when input is tensor([18]), the target is: 47
when input is tensor([18, 47]), the target is: 56
when input is tensor([18, 47, 56]), the target is: 57
when input is tensor([18, 47, 56, 57]), the target is: 58
when input is tensor([18, 47, 56, 57, 58]), the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is: 58


In [33]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(train = True):
    data = train_data if train else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch()
print('inputs')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)

inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [53]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from typing import Optional

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    # idx and targets are both (batch_size, block_size) tensors of integers
    def forward(self, idx, targets = None) -> tuple[torch.Tensor, Optional[torch.Tensor]]:
        logits = self.token_embedding_table(idx) # (B=batch_size, T=block_size, C=vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))
        
        return logits, loss

    def generate(self, idx, max_new_tokens) -> torch.Tensor:
        # idx is (B, T) array of indexes
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m.forward(xb, yb)

print(logits.shape)
print(loss)

print(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100))
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 8, 65])
tensor(4.7863, grad_fn=<NllLossBackward0>)
tensor([[ 0, 31, 23, 21, 41, 24, 32, 11, 13, 41, 17, 24, 25, 53, 32, 40, 60, 38,
         60,  1, 15, 12, 52, 55,  7, 29, 17,  9,  9, 10, 15, 22, 55, 49, 27, 23,
         20,  7, 55, 11, 10, 50, 39,  2, 53, 47, 63, 61, 49, 20, 48, 45, 15, 46,
         64, 40, 29, 12, 59,  2,  9, 40, 24, 21, 45, 61, 43, 60, 51, 63, 18, 22,
         19, 33, 19, 54,  0, 61, 52, 37, 35, 51, 52, 62, 23, 35, 35, 43, 60,  7,
         58, 16, 55, 36, 17, 56, 34, 23, 24, 45, 22]])

pJ:Bpm&yiltNCjeO3:Cx&vvMYW-txjuAd IRFbTpJ$zkZelxZtTlHNzdXXUiQQY:qFINTOBNLI,&oTigq z.c:Cq,SDXzetn3XVj


In [47]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
for steps in range(1000):
    xb, yb = get_batch(True)
    logits, loss = m.forward(xb, yb)

    if loss is not None:    
        optimizer.zero_grad(True)
        loss.backward()
        optimizer.step()
        print(loss.item())

2.674377202987671
2.7571005821228027
2.693875789642334
2.6785035133361816
2.629836320877075
2.7569453716278076
2.6945292949676514
2.7265214920043945
2.622645616531372
2.6339926719665527
2.6342153549194336
2.8628621101379395
2.767258405685425
2.631242513656616
2.7720959186553955
2.680079698562622
2.6764140129089355
2.6853339672088623
2.6356284618377686
2.7522470951080322
2.81662917137146
2.7119219303131104
2.6986587047576904
2.719151020050049
2.704956531524658
2.6865909099578857
2.7180473804473877
2.643030881881714
2.7090351581573486
2.6478304862976074
2.724590301513672
2.7488956451416016
2.6174917221069336
2.7307262420654297
2.8322904109954834
2.7260146141052246
2.883455276489258
2.556656837463379
2.717517852783203
2.6315858364105225
2.6851823329925537
2.6607236862182617
2.6969211101531982
2.686380386352539
2.8017868995666504
2.786547899246216
2.696150779724121
2.676715850830078
2.6552531719207764
2.7530744075775146
2.760712146759033
2.725825071334839
2.692159414291382
2.62510418891906

In [49]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


He wKSIGRincy;
I e Y:
fthe
IULI y bcoothe t thepl' Is hepenis: m than VZare bmang INaks d k'thistoDUCO:blk go?bin iglowe rwndowy, alis f win thesom!
o s s wa-vef chthjF ite.

Whan,
LUATAd itull
ANGLHxairat finr INond.
It idedladevine.
be yt O'd cy bofreakit spaber

SPOld ber IIthan;stull3The s t oithe ishthesplat?wout, ithe, t be by 'e 'Tqike BA lonod 'T:
Wk$CUMl. thralinin,f-wemat,
Wit muth fis, st IfeneZA:3Thoulof y h hrs govWay eaur!



Or aths; ichatethamyait slise IO mawwTO, himun.

hst s,



In [54]:
eval_iters = 1000

def estimate_loss():
    out = {}
    m.eval()
    for split in [True, False]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xs, ys = get_batch(split)
            logits, loss = m.forward(xs, ys)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out